# BERT finetuning on AG_news-4

In [1]:
# !pip install transformers
# !pip install datasets
# !pip install ipywidgets

## Librairy

In [2]:
import os
import time
import pickle

import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from transformers import BertTokenizer, BertTokenizerFast
from transformers import BertForSequenceClassification, AdamW
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from transformers.data.data_collator import DataCollatorWithPadding

from datasets import load_dataset, Dataset, concatenate_datasets

In [3]:
# print(torch.__version__)
# print(torch.cuda.device_count())
# print(torch.cuda.is_available())
# print(torch.cuda.get_device_name(0))

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# if torch.cuda.is_available():
#     torch.set_default_tensor_type('torch.cuda.FloatTensor')

device

device(type='cuda')

## Global variables

In [4]:
BATCH_SIZE = 24 # cf. paper Sun et al.
NB_EPOCHS = 4   # cf. paper Sun et al.

In [5]:
# CURRENT_PATH = '~/Results/BERT_finetune' # fake path for deployment
CURRENT_PATH = '/raid/home/jeremiec/Ax_results/BERT_finetune_v2'

In [6]:
RESULTS_FILE = os.path.join(CURRENT_PATH, 'ag_news_results.pkl')
RESULTS_DIR = os.path.join(CURRENT_PATH,'ag_news/')

In [7]:
# CACHE_DIR = '~/Data/huggignface/'         # path of your folder
CACHE_DIR = '/raid/home/jeremiec/huggingface_datasets'

## Dataset

In [8]:
# download dataset

raw_datasets = load_dataset('ag_news', cache_dir=CACHE_DIR)

Downloading:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

Using custom data configuration default
Reusing dataset ag_news (/raid/home/jeremiec/huggingface_datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
# tokenize

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

train_dataset = tokenized_datasets["train"].shuffle(seed=42)
train_val_datasets = train_dataset.train_test_split(train_size=0.8)

train_dataset = train_val_datasets['train'].rename_column('label', 'labels')
val_dataset = train_val_datasets['test'].rename_column('label', 'labels')
test_dataset = tokenized_datasets["test"].shuffle(seed=42).rename_column('label', 'labels')

  0%|          | 0/120 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [10]:
# get number of labels

num_labels = len(set(train_dataset['labels'].tolist()))
num_labels

4

## Model

#### Model

In [11]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

#### Training

In [12]:
training_args = TrainingArguments(
    
    # output
    output_dir=RESULTS_DIR,          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # cf. paper Sun et al.
    learning_rate=2e-5,                       # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
#     # eval
    evaluation_strategy="steps",              # cf. paper Sun et al.
    eval_steps=50, # 20                       # cf. paper Sun et al.
#     evaluation_strategy='no', # no more evaluation, takes time
    
    # log
    logging_dir=RESULTS_DIR+'logs',  
    logging_strategy='steps',
    logging_steps=50, # 20
    
    # save
    save_strategy='steps',
    save_total_limit=1,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    metric_for_best_model='eval_loss'    
)

In [13]:
def compute_metrics(p):
    
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    
    return {"val_accuracy": accuracy}

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    # compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [ ]:
results = trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 96000
  Num Epochs = 4
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 16000


Step,Training Loss,Validation Loss
50,1.403400,1.394444
100,1.379600,1.352528
150,1.301800,1.231805
200,1.167500,1.047616
250,0.980300,0.834616
300,0.739400,0.616660
350,0.547400,0.437883
400,0.447100,0.370030
450,0.402600,0.343805
500,0.341300,0.313804


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 24000
  Batch size = 24
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 24000
  Batch size = 24
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 24000
  Batch size = 24
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 24000
  Batch size = 24
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequence

In [ ]:
training_time = results.metrics["train_runtime"]
training_time_per_epoch = training_time / training_args.num_train_epochs
training_time_per_epoch

In [ ]:
trainer.save_model(os.path.join(RESULTS_DIR, 'checkpoint_best_model'))

## Results

In [ ]:
# load model
model_file = os.path.join(RESULTS_DIR, 'checkpoint_best_model')
finetuned_model = BertForSequenceClassification.from_pretrained(model_file, num_labels=num_labels)
finetuned_model.to(device)
finetuned_model.eval()

# compute test acc
test_trainer = Trainer(finetuned_model, data_collator=DataCollatorWithPadding(tokenizer))
raw_preds, labels, _ = test_trainer.predict(test_dataset)
preds = np.argmax(raw_preds, axis=1)
test_acc = accuracy_score(y_true=labels, y_pred=preds)

# save results
results_d = {}
results_d['accuracy'] = test_acc
results_d['training_time'] = training_time

In [22]:
results_d

{'accuracy': 0.9406578947368421, 'training_time': 62729.3993}

In [ ]:
# save results

with open(RESULTS_FILE, 'wb') as fh:
    pickle.dump(results_d, fh)